In [1]:
import os, glob
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageOps, ImageFile
import matplotlib.pyplot as plt

In [2]:
def mkfolder(folder):
    if not os.path.lexists(folder):
        os.makedirs(folder)
    
def imshow_plt(img):
    plt.figure(figsize=(10,10))
    plt.imshow(img, cmap ='gray')
    plt.show()
    
def weight_center(img):
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    cnt = contours[0] 
#     print(cnt)
    mmt = cv2.moments(cnt) 
#     for key, value in mmt.items(): 
#         print(key," : ",value) 
    cx = int(mmt['m10']/mmt['m00']) 
    cy = int(mmt['m01']/mmt['m00']) 
    print( 'x 무게중심', cx, 'y 무게중심', cy )
    return cx, cy


In [3]:
def randomRoate(img, label, p, angle_range):
    if len(img.shape)==3:
        height, width, channel = img.shape
    else:
        height, width = img.shape
    
    random_angle = np.random.randint(-angle_range/2, angle_range/2)*2
#     print(random_angle)
    matrix = cv2.getRotationMatrix2D((width/2, height/2), random_angle, 1)
    rotate_img = cv2.warpAffine(img, matrix, (width, height))
    rotate_label = cv2.warpAffine(label, matrix, (width, height))
#     imshow_plt(rotate_img)
#     imshow_plt(rotate_label)
    
    return rotate_img, rotate_label

def find_top_point(ori_img, mask_img):
    mask_img_T = mask_img.T

    tmp_index=[]
    for i, c in enumerate(mask_img_T):
        if len(np.unique(c))>1:
            tmp_index.append(i)
    x_min = min(tmp_index)
    x_max = max(tmp_index)
#     print(x_min, x_max)

    p_x = int((x_min+x_max)/2) 
    p_y = np.where(mask_img_T[int((x_min+x_max)/2)]==255)[0][0]
    p = (p_x, p_y)


#     cv2.line(ori_img, p, p, (255,0,0), 5)
#     imshow_plt(ori_img)
    
    return p_x, p_y

In [4]:
img_path = '../3_deepdata/1_exp1/train/AP_pre_img/'
mask_path = '../3_deepdata/1_exp1/train/AP_pre_label/'

img_li = sorted(glob.glob(img_path+'*.png'))
mask_li = sorted(glob.glob(mask_path+'*png'))

print(len(img_li), len(mask_li))

1014 1014


In [5]:
def Augment_crop(img, mask):
    p_x, p_y=find_top_point(ori_img, mask_img)
    rotate_img, rotate_mask = randomRoate(img, mask, (p_x, p_y), 20)
    random_size = np.random.randint(15,35)*20 # 300-600
#     print(random_size)
    
    h, w= img.shape
    x1 = p_x-random_size if p_x-random_size>0 else 0
    x2 = p_x+random_size if p_x+random_size<w else w
    y1 = p_y-random_size if p_y-random_size>0 else 0
    y2 = p_y+random_size if p_y+random_size<h else h
    
#     print(h,w)
#     print(x1,x2,y1,y2)
    crop_img = rotate_img[y1:y2,x1:x2]
    crop_mask = rotate_mask[y1:y2,x1:x2]
    
#     imshow_plt(crop_img)
#     imshow_plt(crop_mask)
#     print(crop_mask.shape)
#     if crop_img==[] or crop_mask==[]:
#         imshow_plt(ori_img)
        
    return crop_img, crop_mask

In [6]:
i=0

mkfolder('../3_deepdata/1_exp1/train/AP_aug_img/')
mkfolder('../3_deepdata/1_exp1/train/AP_aug_label/')
for img, mask in zip(img_li, mask_li):
#     if i==10:
#         break
    if i%100==0:
        print('{}/{}'.format(i, len(img_li)))
        
    ori_img = cv2.imread(img, 0)
    mask_img = cv2.imread(mask, 0)
    img_name = img[img.rindex('/')+1:-4]
#     print(img_name)
    
    cv2.imwrite('../3_deepdata/1_exp1/train/AP_aug_img/{}.png'.format(img_name), cv2.resize(ori_img, (512,512)))
    cv2.imwrite('../3_deepdata/1_exp1/train/AP_aug_label/{}.png'.format(img_name), cv2.resize(mask_img, (512,512)))
    for j in range(9):
        aug_img, aug_mask = Augment_crop(ori_img, mask_img)        
        aug_img = cv2.resize(aug_img, (512,512))
        aug_mask = cv2.resize(aug_mask, (512,512))
        cv2.imwrite('../3_deepdata/1_exp1/train/AP_aug_img/{}_{}.png'.format(img_name, j), aug_img)
        cv2.imwrite('../3_deepdata/1_exp1/train/AP_aug_label/{}_{}.png'.format(img_name, j), aug_mask)
#     imshow_plt(ori_img)
    
    i+=1

0/1014
10/1014
20/1014
30/1014
40/1014
50/1014
60/1014
70/1014
80/1014
90/1014
100/1014
110/1014
120/1014
130/1014
140/1014
150/1014
160/1014
170/1014
180/1014
190/1014
200/1014
210/1014
220/1014
230/1014
240/1014
250/1014
260/1014
270/1014
280/1014
290/1014
300/1014
310/1014
320/1014
330/1014
340/1014
350/1014
360/1014
370/1014
380/1014
390/1014
400/1014
410/1014
420/1014
430/1014
440/1014
450/1014
460/1014
470/1014
480/1014
490/1014
500/1014
510/1014
520/1014
530/1014
540/1014
550/1014
560/1014
570/1014
580/1014
590/1014
600/1014
610/1014
620/1014
630/1014
640/1014
650/1014
660/1014
670/1014
680/1014
690/1014
700/1014
710/1014
720/1014
730/1014
740/1014
750/1014
760/1014
770/1014
780/1014
790/1014
800/1014
810/1014
820/1014
830/1014
840/1014
850/1014
860/1014
870/1014
880/1014
890/1014
900/1014
910/1014
920/1014
930/1014
940/1014
950/1014
960/1014
970/1014
980/1014
990/1014
1000/1014
1010/1014
